# Building Our Dataset

The flow for data labeling:
1. Seperate drum track from original selected song.
2. For each drum track, apply onset detection using Librosa to extract each hit
3. Use `pigeon` to display the options and apply annotation
4. At the end, save our results into a JSON file for record.

In [1]:
import librosa
from IPython.display import Audio

import numpy as np

import json
from datetime import datetime as dt
import os

from utils.audio_utils import fix_audio_length

## Creating a Session Class

A session object represents a labeling session, which consists of:
- A drum track
- An index for checkpointing purposes
- Annotations made
- A save checkpoint function to save annotations and checkpoint index

In [2]:
from utils.forked_pigeon import annotate
from utils.config import SETTINGS

class LabellingSession():
    def __init__(self, audio_path: str, annotations:list = [], starting_index:int = 0):
        self.audio_path = audio_path
        self.annotations = annotations
        self.checkpoint = starting_index
        
        samples, sr = librosa.load(audio_path, sr=44100)
        onset_frames = librosa.onset.onset_detect(y=samples, sr=sr, 
                    units='samples', backtrack=True)
        
        self.samples = samples
        self.sr = sr
        self.hits = list(zip(onset_frames[:-1], onset_frames[1:]))
        

    def begin(self) -> list:
        """ Creates an annotation loop, ends with returning annotations
        """
        try:
            hits = self.hits[self.checkpoint:] # start from where we ended off

            labelling_examples = [fix_audio_length(self.samples[h[0]: h[1]], 
                                                   self.sr, 1) 
                                  for h in hits]

            annotations, checkpoint_index = annotate(self.audio_path, labelling_examples,
              options=list(SETTINGS['LABELS_INDEX'].values()),
              display_fn=lambda example: display(Audio(example, rate=self.sr, autoplay=True))
            )

            self.annotations = annotations
            self.checkpoint = checkpoint_index
        
        except Exception as e:
            print(e)
            self.checkpoint = len(annotations)
            self.save_checkpoint()
            

    def save_checkpoint(self):
        checkpoint = {
            'date': dt.now().strftime("%Y-%m-%d"),
            'index_up_to': self.checkpoint,
            'annotations': self.annotations
        }
                
        json_object = json.dumps(checkpoint)
        
        with open(f'./dataset/labels/{dt.now().strftime("%Y_%m_%d_%H_%M_%S")}.json', "w") as outfile:
            outfile.write(json_object)
        
        print('Saved.')
    

In [3]:
session = LabellingSession(audio_path='./drum_tracks/1.mp3')

session.begin()

HTML(value='0 examples annotated, 356 examples left')

Output()

In [18]:
session.save_checkpoint()

Saved.


### Label Summary

After labelling, below is a summary of all the labels we've done.

In [4]:
import os

label_files = os.listdir("./labels")

count_dict = {label: 0 for label in list(SETTINGS['LABELS_INDEX'].values())}

for file in label_files:
    f = open(f"./labels/{file}", "r")
    
    annotations = json.load(f).get('annotations')
    annotations = [a[1] for a in annotations] # just take the labels instead of the path
    
    for label in count_dict.keys():
        c = annotations.count(label)
        count_dict[label] += c
    
    
print(f"Label Counts: {count_dict}")

Label Counts: {'crash': 352, 'hihat_c': 485, 'kick_drum': 1512, 'ride': 351, 'snare': 1576, 'tom_h': 237}
